In [1]:
import openai
import json

In [2]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [3]:
openai_api_key = config.get("OPENAI_API_KEY")

In [4]:
openai.api_key = openai_api_key

In [5]:
from IPython.display import Markdown, display

def display_colors(colors):
    display(Markdown(" ".join(
        f"<span style='color:{color}'>{chr(9608)*4}</span>" for color in colors)
        ))

In [6]:
import re

def get_colors_from_string(string):
    return re.findall(r'#[0-9a-fA-F]{6}', string)

In [63]:
def get_and_render_colors(msg):
    prompt = f"""
    ### Instructions ###
    You are a color patte generating assistant that reponds to text prompts for color palettes.
    You should generate color palettes that fit the theme, mood, or instrauctions in the prompt.
    The palettes should be between 2 to 8 colors.

    Input: {msg}

    Desired format: a JSON array of hexagonal color codes such as ["#FF0000", "#00FF00", "#0000FF"]

    Result:
    """

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=60
    )

    colors = get_colors_from_string(response['choices'][0]['text'])

    display_colors(colors)

    return response, colors


response, colors = get_and_render_colors("I feel crazy today. I want a color palette that reflects my mood.")

<span style='color:#FE4A49'>████</span> <span style='color:#FE9F80'>████</span> <span style='color:#D81159'>████</span> <span style='color:#7D1E7D'>████</span> <span style='color:#580DA6'>████</span> <span style='color:#5C4742'>████</span> <span style='color:#9295CA'>████</span>

In [64]:
response

<OpenAIObject text_completion id=cmpl-7SfXp6wtlKQKrYa3erYGujq1xhi1b at 0x2a245e1c230> JSON: {
  "id": "cmpl-7SfXp6wtlKQKrYa3erYGujq1xhi1b",
  "object": "text_completion",
  "created": 1687067237,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": " [\"#FE4A49\", \"#FE9F80\", \"#D81159\", \"#7D1E7D\", \"#580DA6\", \"#5C4742\", \"#9295CA\"]",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 118,
    "completion_tokens": 43,
    "total_tokens": 161
  }
}

## Discussion:

Response can be unexpected with the given hexagonal color codes but also some unexpected string with it. The best way is to use RE to filter down to the desired colors we want.

In [39]:
colors = json.loads(response['choices'][0]['text'])

In [41]:

display_colors(colors)

<span style='color:#D22A54'>████</span> <span style='color:#F1C38E'>████</span> <span style='color:#EEC68E'>████</span> <span style='color:#B6AD27'>████</span> <span style='color:#C8B6AA'>████</span>

## Rewrite with ChatCompletion

In [14]:
def get_and_render_colors_ChatCompletion(msg):
    system_prompt = f"""
    ### Instructions ###
    You are a color patte generating assistant that reponds to text prompts for color palettes.
    You should generate color palettes that fit the theme, mood, or instrauctions in the prompt.
    The palettes should be between 2 to 8 colors.

    Desired format: a JSON array of hexagonal color codes such as ["#FF0000", "#00FF00", "#0000FF"]
    """

    user_prompt = f"""
    User's input: {msg}
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=60
    )

    colors = get_colors_from_string(response['choices'][0]['message']['content'])

    display_colors(colors)

    return response, colors


response, colors = get_and_render_colors_ChatCompletion("A wonderful day in Perth in winter")

<span style='color:#0072bb'>████</span> <span style='color:#a8cddd'>████</span> <span style='color:#d4e5e1'>████</span> <span style='color:#8cb8b3'>████</span>

In [13]:
print(response)

{
  "id": "chatcmpl-7T8YouXONwV9GVidI0n1Tfpwl5I9m",
  "object": "chat.completion",
  "created": 1687178774,
  "model": "gpt-3.5-turbo-0301",
  "usage": {
    "prompt_tokens": 128,
    "completion_tokens": 60,
    "total_tokens": 188
  },
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "Here is a color palette that reflects a crazy mood:\n\n[\"#FF4136\", \"#FFDC00\", \"#0074D9\", \"#B10DC9\"] \n\nThis palette has bold and lively colors that could represent the energy and excitement of feeling crazy. The bright red and yellow colors signify"
      },
      "finish_reason": "length",
      "index": 0
    }
  ]
}
